# Generative Adversarial Networks
### Brandon Yeu

1. GAN training is modeled as a two-player minimax game between a generator, which tries to produce fake data that appears real, and a discriminator, which tries to identify real or fake data. The generator's goal is to minimize the discriminator's success, but the discriminator's goal is to maximize its success.

2. GANs are powerful, but it can often be difficult to train them. One common challenge is mode collapse, in which the generator produces a limited variety of outputs, as opposed to a diverse set of realistic data. This can cause the model to map different latent inputs to the same/similar samples. Mode collapse can occur if the reward function is not diverse or if the generator finds a quick way to exploit the discriminator. Techniques such as batch normalization, minibatch discrimination, and Wassertein GAN can help mitigate mode collapse.

3. In an adversarial network, the discriminator is an adaptive loss function for the generator. It helps to dynamically train the model, and it improves with the generator continuously redefining how they function during training.

4. Inception score (IS) evaluates GAN performances by rewarding high confidence predictions and high diversity, but it ignores real data distribution, can be bypassed by sharp, but unrealistic images, and is sensitive to the pretrained classifier. Fréchet inception distance (FID) evaluates GAN performances by comparing statistics of real and generated images in feature space. FID measures image quality compared to real images and diversity, so it better correlates with human comparison and penalizes mode collapse.

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

# Load and preprocess data
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize to [-1, 1]
BUFFER_SIZE = 60000
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Generator model
def make_generator_model():
  model = tf.keras.Sequential([
      layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
      layers.BatchNormalization(),
      layers.LeakyReLU(),
      layers.Reshape((7, 7, 256)),
      layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.LeakyReLU(),
      layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.LeakyReLU(),
      layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
  ])
  return model

# Discriminator model
def make_discriminator_model():
  model = tf.keras.Sequential([
      layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
      layers.LeakyReLU(),
      layers.Dropout(0.3),
      layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
      layers.LeakyReLU(),
      layers.Dropout(0.3),
      layers.Flatten(),
      layers.Dense(1)
  ])
  return model

# Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  return real_loss + fake_loss

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator = make_generator_model()
discriminator = make_discriminator_model()
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Training function
@tf.function
def train_step(images):
  noise = tf.random.normal([BATCH_SIZE, 100])
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training=True)
    real_output = discriminator(images, training=True)
    fake_output = discriminator(generated_images, training=True)
    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Training loop
def train(dataset, epochs):
  for epoch in range(epochs):
    for image_batch in dataset:
      train_step(image_batch)

# Run the training
train(train_dataset, epochs=1)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import time
import numpy as np

# Hyperparameters
BUFFER_SIZE = 50000
BATCH_SIZE = 256
EPOCHS = 50
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16

# Load and preprocess data (CIFAR-10)
(train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()

train_images = train_images.astype('float32')
train_images = (train_images - 127.5) / 127.5

train_dataset = tf.data.Dataset.from_tensor_slices(train_images)\
    .shuffle(BUFFER_SIZE)\
    .batch(BATCH_SIZE)

# Generator model
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(4 * 4 * 512, use_bias=False, input_shape=(NOISE_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((4, 4, 512)),

        layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same',
                               use_bias=False, activation='tanh')
    ])
    return model

# Discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                      input_shape=[32, 32, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

# Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

# Training function
@tf.function
def train_step(images):
    noise = tf.random.normal([tf.shape(images)[0], NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(
        gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(
        disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(
        zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(
        zip(gradients_of_discriminator, discriminator.trainable_variables))

# Image saving
seed = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])
os.makedirs("generated_images", exist_ok=True)

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(f'generated_images/image_at_epoch_{epoch:04d}.png')
    plt.close()

# Training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        if (epoch + 1) % 10 == 0:
            generate_and_save_images(generator, epoch + 1, seed)

        print(f'Epoch {epoch + 1}, Time: {time.time() - start:.2f} sec')

    generate_and_save_images(generator, epochs, seed)

# Run training
train(train_dataset, EPOCHS)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1, Time: 50.32 sec
Epoch 2, Time: 38.95 sec
Epoch 3, Time: 39.20 sec
Epoch 4, Time: 38.85 sec
Epoch 5, Time: 39.34 sec
Epoch 6, Time: 38.97 sec
Epoch 7, Time: 38.98 sec
Epoch 8, Time: 39.04 sec
Epoch 9, Time: 39.08 sec
Epoch 10, Time: 39.64 sec
Epoch 11, Time: 39.13 sec
Epoch 12, Time: 39.15 sec
Epoch 13, Time: 39.20 sec
Epoch 14, Time: 39.02 sec
Epoch 15, Time: 39.21 sec
Epoch 16, Time: 39.09 sec
Epoch 17, Time: 38.92 sec
Epoch 18, Time: 38.93 sec
Epoch 19, Time: 38.96 sec
Epoch 20, Time: 39.20 sec
Epoch 21, Time: 38.98 sec
Epoch 22, Time: 39.06 sec
Epoch 23, Time: 39.08 sec
Epoch 24, Time: 39.16 sec
Epoch 25, Time: 39.03 sec
Epoch 26, Time: 38.97 sec
Epoch 27, Time: 38.98 sec
Epoch 28, Time: 39.01 sec
Epoch 29, Time: 39.01 sec
Epoch 30, Time: 39.21 sec
Epoch 31, Time: 39.05 sec
Epoch 32, Time: 39.15 sec
Epoch 33, Time: 38.93 sec
Epoch 34, Time: 39.23 sec
Epoch 35, Time: 39.01 sec
Epoch 36, Time: 39.20 sec
Epoch 37, Time: 39.01 sec
Epoch 38, Time: 40.94 sec
Epoch 39, Time: 39.32